In [1]:
# TODO:
# temporary workaround:
# remove cell before PR
import os
import sys
os.getcwd()
os.chdir("/mnt/ml-team/homes/eryk.mazus/langchain/libs/langchain/")
os.getcwd()

# sys.path.append('/mnt/ml-team/homes/eryk.mazus/langchain/')
sys.path.append('/mnt/ml-team/homes/eryk.mazus/langchain/libs/langchain/')


# LLama Chat

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Hugging Face imports:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
)

# LangChain imports:
from langchain.chat_models import ChatLlama2
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

This notebook assumes that you were granted with access to the Llama 2 models in the Hugging Face models hub. To use the model locally, you need to be [logged in](https://huggingface.co/docs/huggingface_hub/quick-start#login) with a Hugging Face account.

In [4]:
from huggingface_hub import login
login()

In [5]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_4bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
pipe = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [9]:
chat = ChatLlama2(pipeline=pipe)

In [10]:
# Generation kwargs:
pipeline_kwargs = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.7,
    "eos_token_id": tokenizer.eos_token_id,
    "max_length": 256,    
}

### Single calls:

In [11]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chat(messages, **pipeline_kwargs)

AIMessage(content=' Sure! Here is the translation of "I love programming" from English to French:\nJe adore le programming.\n\nI hope this helps! Let me know if you have any other sentences you would like me to translate.', additional_kwargs={}, example=False)

### Batch calls:

In [12]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text=' Great! "Programming" is "le programming" in French.\n\nSo, you love programming? "J\'aime le programming" in French.', generation_info=None, message=AIMessage(content=' Great! "Programming" is "le programming" in French.\n\nSo, you love programming? "J\'aime le programming" in French.', additional_kwargs={}, example=False))], [ChatGeneration(text=" Je suis ravi d'être votre assistant de traduction pour l'anglais à français. Vous pouvez maintenant dire ce que vous voulez en français, et je serai heureux de le traduire pour vous.\nMerci de me dire que vous aimez l'intelligence artificielle. C'est un sujet très intéressant et qui a des applications nombreuses dans différents domaines, tels que la robotique, l'apprentissage automatique, la reconnaissance faciale et de la parole, et bien plus encore.\nPourriez-vous me dire ce que vous voulez savoir sur l'intelligence artificielle? Je suis là pour vous aider et vous fournir des informations préci